In [22]:
import xarray as xr 
import numpy as np

import sys
sys.path.append('/home/users/cturrell/documents/eddy_feedback')
# pylint: disable=wrong-import-position
import functions.data_wrangling as data
import functions.eddy_feedback as ef

## CanESM5

epfy (300)
- EmonZ: 39 plev

ua (300)
- Amon: 19
- AERmon: 39

In [23]:
can_epfy = xr.open_mfdataset('/gws/nopw/j04/arctic_connect/cturrell/PAMIP_data/monthly/pdSST-pdSIC/epfy/CanESM5/*.nc', 
                         combine='nested', concat_dim='ens_ax', parallel=True)
can_epfy 

<xarray.Dataset>
Dimensions:    (time: 14, ens_ax: 300, bnds: 2, plev: 39, lat: 64)
Coordinates:
  * time       (time) object 2000-04-16 00:00:00 ... 2001-05-16 12:00:00
  * plev       (plev) float64 1e+05 9.25e+04 8.5e+04 7e+04 ... 10.0 7.0 5.0 3.0
  * lat        (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 82.31 85.1 87.86
Dimensions without coordinates: ens_ax, bnds
Data variables:
    time_bnds  (ens_ax, time, bnds) object dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    lat_bnds   (ens_ax, lat, bnds) float64 dask.array<chunksize=(1, 64, 2), meta=np.ndarray>
    epfy       (ens_ax, time, plev, lat) float32 dask.array<chunksize=(1, 1, 39, 64), meta=np.ndarray>
Attributes: (12/52)
    CCCma_model_hash:            37a65f96056a7aa4fa354897986cc70301801f67
    CCCma_parent_runid:          p2-amip-ru10
    CCCma_pycmor_hash:           8157ca33437ab096d583668ba0da0c2ce55acfe1
    CCCma_runid:                 pamip11-100
    Conventions:                 CF-1.7 CMIP-6.2
    YMDH_branch_time_in_child:   2000:04:01:00
    ...                          ...
    tracking_id:                 hdl:21.14100/f30700de-4b1e-47b8-b50a-82c5595...
    variable_id:                 epfy
    variant_label:               r100i1p2f1
    version:                     v20190429
    license:                     CMIP6 model data produced by The Government ...
    cmor_version:                3.4.0

In [24]:
can_ua = xr.open_mfdataset('/gws/nopw/j04/arctic_connect/cturrell/PAMIP_data/monthly/pdSST-pdSIC/ua/CanESM5/Amon/*.nc', 
                         combine='nested', concat_dim='ens_ax', parallel=True)

# can_ua = xr.open_mfdataset('/gws/nopw/j04/arctic_connect/cturrell/PAMIP_data/monthly/pdSST-pdSIC/ua/CanESM5/AERmon/*.nc', 
#                          combine='nested', concat_dim='ens_ax', parallel=True)
can_ua

<xarray.Dataset>
Dimensions:    (time: 14, ens_ax: 300, bnds: 2, plev: 19, lat: 64, lon: 128)
Coordinates:
  * time       (time) object 2000-04-16 00:00:00 ... 2001-05-16 12:00:00
  * plev       (plev) float64 1e+05 9.25e+04 8.5e+04 7e+04 ... 1e+03 500.0 100.0
  * lat        (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 82.31 85.1 87.86
  * lon        (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
Dimensions without coordinates: ens_ax, bnds
Data variables:
    time_bnds  (ens_ax, time, bnds) object dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    lat_bnds   (ens_ax, lat, bnds) float64 dask.array<chunksize=(1, 64, 2), meta=np.ndarray>
    lon_bnds   (ens_ax, lon, bnds) float64 dask.array<chunksize=(1, 128, 2), meta=np.ndarray>
    ua         (ens_ax, time, plev, lat, lon) float32 dask.array<chunksize=(1, 1, 19, 64, 128), meta=np.ndarray>
Attributes: (12/53)
    CCCma_model_hash:            37a65f96056a7aa4fa354897986cc70301801f67
    CCCma_parent_runid:          p2-amip-ru10
    CCCma_pycmor_hash:           8157ca33437ab096d583668ba0da0c2ce55acfe1
    CCCma_runid:                 pamip11-100
    Conventions:                 CF-1.7 CMIP-6.2
    YMDH_branch_time_in_child:   2000:04:01:00
    ...                          ...
    tracking_id:                 hdl:21.14100/a19bb4fe-cb57-40d2-b524-2292caf...
    variable_id:                 ua
    variant_label:               r100i1p2f1
    version:                     v20190429
    license:                     CMIP6 model data produced by The Government ...
    cmor_version:                3.4.0

### Eddy Feedback Parameter

In [25]:
can_epfy = can_epfy.sel( plev = can_ua.plev.values )

can_ds = xr.Dataset( {'ubar': can_ua.ua.mean('lon'), 'div1': can_epfy.epfy})
can_ds = can_ds.rename({'plev': 'level'})
can_ds

<xarray.Dataset>
Dimensions:  (time: 14, level: 19, lat: 64, ens_ax: 300)
Coordinates:
  * time     (time) object 2000-04-16 00:00:00 ... 2001-05-16 12:00:00
  * level    (level) float64 1e+05 9.25e+04 8.5e+04 7e+04 ... 1e+03 500.0 100.0
  * lat      (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 79.53 82.31 85.1 87.86
Dimensions without coordinates: ens_ax
Data variables:
    ubar     (ens_ax, time, level, lat) float32 dask.array<chunksize=(1, 1, 19, 64), meta=np.ndarray>
    div1     (ens_ax, time, level, lat) float32 dask.array<chunksize=(1, 1, 19, 64), meta=np.ndarray>

In [26]:
can_efp = ef.calculate_efp_pamip(can_ds)
can_efp

0.48

# MIROC6

In [27]:
miroc_epfy = xr.open_mfdataset('/gws/nopw/j04/arctic_connect/cturrell/PAMIP_data/monthly/pdSST-pdSIC/epfy/MIROC6/*.nc', 
                         combine='nested', concat_dim='ens_ax', parallel=True)
miroc_epfy 

<xarray.Dataset>
Dimensions:    (time: 12, ens_ax: 100, bnds: 2, plev: 39, lat: 128)
Coordinates:
  * time       (time) datetime64[ns] 2000-06-16 ... 2001-05-16T12:00:00
  * plev       (plev) float64 1e+05 9.25e+04 8.5e+04 7e+04 ... 10.0 7.0 5.0 3.0
  * lat        (lat) float64 -88.93 -87.54 -86.14 -84.74 ... 86.14 87.54 88.93
Dimensions without coordinates: ens_ax, bnds
Data variables:
    time_bnds  (ens_ax, time, bnds) datetime64[ns] dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    lat_bnds   (ens_ax, lat, bnds) float64 dask.array<chunksize=(1, 128, 2), meta=np.ndarray>
    epfy       (ens_ax, time, plev, lat) float32 dask.array<chunksize=(1, 1, 39, 128), meta=np.ndarray>
Attributes: (12/43)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            PAMIP
    branch_method:          standard
    branch_time_in_child:   -91.0
    branch_time_in_parent:  4018.0
    creation_date:          2019-09-30T04:01:59Z
    ...                     ...
    title:                  MIROC6 output prepared for CMIP6
    variable_id:            epfy
    variant_label:          r100i1p1f1
    license:                CMIP6 model data produced by MIROC is licensed un...
    cmor_version:           3.4.0
    tracking_id:            hdl:21.14100/349bd45f-b3cb-4fa0-92c7-985b26ffcf7b

In [28]:
miroc_ua = xr.open_mfdataset('/gws/nopw/j04/arctic_connect/cturrell/PAMIP_data/monthly/pdSST-pdSIC/ua/MIROC6/*.nc', 
                         combine='nested', concat_dim='ens_ax', parallel=True)
miroc_ua 

<xarray.Dataset>
Dimensions:    (time: 12, ens_ax: 100, bnds: 2, plev: 19, lat: 128, lon: 256)
Coordinates:
  * time       (time) datetime64[ns] 2000-06-16 ... 2001-05-16T12:00:00
  * plev       (plev) float64 1e+05 9.25e+04 8.5e+04 7e+04 ... 1e+03 500.0 100.0
  * lat        (lat) float64 -88.93 -87.54 -86.14 -84.74 ... 86.14 87.54 88.93
  * lon        (lon) float64 0.0 1.406 2.812 4.219 ... 354.4 355.8 357.2 358.6
Dimensions without coordinates: ens_ax, bnds
Data variables:
    time_bnds  (ens_ax, time, bnds) datetime64[ns] dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    lat_bnds   (ens_ax, lat, bnds) float64 dask.array<chunksize=(1, 128, 2), meta=np.ndarray>
    lon_bnds   (ens_ax, lon, bnds) float64 dask.array<chunksize=(1, 256, 2), meta=np.ndarray>
    ua         (ens_ax, time, plev, lat, lon) float32 dask.array<chunksize=(1, 1, 19, 128, 256), meta=np.ndarray>
Attributes: (12/45)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            PAMIP
    branch_method:          standard
    branch_time_in_child:   -91.0
    branch_time_in_parent:  4018.0
    comment:                The first 40 members branched off from AMIP exper...
    ...                     ...
    title:                  MIROC6 output prepared for CMIP6
    variable_id:            ua
    variant_label:          r100i1p1f1
    license:                CMIP6 model data produced by MIROC is licensed un...
    cmor_version:           3.4.0
    tracking_id:            hdl:21.14100/716d1b78-797e-405a-bc0b-0f1892669912

### Eddy Feedback Parameter

In [29]:
miroc_epfy = miroc_epfy.sel( plev = miroc_ua.plev.values )

miroc_ds = xr.Dataset( {'ubar': miroc_ua.ua.mean('lon'), 'div1': miroc_epfy.epfy})
miroc_ds = miroc_ds.rename({'plev': 'level'})
miroc_ds

<xarray.Dataset>
Dimensions:  (time: 12, level: 19, lat: 128, ens_ax: 100)
Coordinates:
  * time     (time) datetime64[ns] 2000-06-16 ... 2001-05-16T12:00:00
  * level    (level) float64 1e+05 9.25e+04 8.5e+04 7e+04 ... 1e+03 500.0 100.0
  * lat      (lat) float64 -88.93 -87.54 -86.14 -84.74 ... 86.14 87.54 88.93
Dimensions without coordinates: ens_ax
Data variables:
    ubar     (ens_ax, time, level, lat) float32 dask.array<chunksize=(1, 1, 19, 128), meta=np.ndarray>
    div1     (ens_ax, time, level, lat) float32 dask.array<chunksize=(1, 1, 19, 128), meta=np.ndarray>

In [30]:
miroc_efp = ef.calculate_efp_pamip(miroc_ds)
miroc_efp

/home/users/cturrell/miniforge3/envs/eddy/lib/python3.12/site-packages/dask/array/numpy_compat.py:51: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)


0.03

## NorESM2-LM

epfy (200)
- EmonZ: 39 plev

ua (200)
- Emon: 9 plev
- Amon: 19
- AERmon: 32

In [31]:
nor_epfy = xr.open_mfdataset('/gws/nopw/j04/arctic_connect/cturrell/PAMIP_data/monthly/pdSST-pdSIC/epfy/NorESM2-LM/*.nc', 
                         combine='nested', concat_dim='ens_ax', parallel=True)
nor_epfy

<xarray.Dataset>
Dimensions:    (time: 12, ens_ax: 200, bnds: 2, plev: 39, lat: 96)
Coordinates:
  * time       (time) object 2000-06-16 00:00:00 ... 2001-05-16 12:00:00
  * plev       (plev) float64 1e+05 9.25e+04 8.5e+04 7e+04 ... 10.0 7.0 5.0 3.0
  * lat        (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
Dimensions without coordinates: ens_ax, bnds
Data variables:
    time_bnds  (ens_ax, time, bnds) object dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    lat_bnds   (ens_ax, lat, bnds) float64 dask.array<chunksize=(1, 96, 2), meta=np.ndarray>
    epfy       (ens_ax, time, plev, lat) float32 dask.array<chunksize=(1, 1, 39, 96), meta=np.ndarray>
Attributes: (12/48)
    Conventions:               CF-1.7 CMIP-6.2
    activity_id:               PAMIP
    branch_method:             Hybrid-restart from year 2000-04-01 of amip
    branch_time:               0.0
    branch_time_in_child:      0.0
    branch_time_in_parent:     9215.0
    ...                        ...
    title:                     NorESM2-LM output prepared for CMIP6
    variable_id:               epfy
    variant_label:             r100i1p1f1
    license:                   CMIP6 model data produced by NCC is licensed u...
    cmor_version:              3.5.0
    tracking_id:               hdl:21.14100/43baa687-cf69-4640-b923-613139c0e245

In [32]:
# nor_ua = xr.open_mfdataset('/gws/nopw/j04/arctic_connect/cturrell/PAMIP_data/monthly/pdSST-pdSIC/ua/NorESM2-LM/Emon/*.nc', 
#                          combine='nested', concat_dim='ens_ax', parallel=True)

nor_ua = xr.open_mfdataset('/gws/nopw/j04/arctic_connect/cturrell/PAMIP_data/monthly/pdSST-pdSIC/ua/NorESM2-LM/Amon/*.nc', 
                         combine='nested', concat_dim='ens_ax', parallel=True, chunks={'time':12})

# nor_ua = xr.open_mfdataset('/gws/nopw/j04/arctic_connect/cturrell/PAMIP_data/monthly/pdSST-pdSIC/ua/NorESM2-LM/AERmon/*.nc', 
#                          combine='nested', concat_dim='ens_ax', parallel=True)
nor_ua

<xarray.Dataset>
Dimensions:    (time: 12, ens_ax: 200, bnds: 2, plev: 19, lat: 96, lon: 144)
Coordinates:
  * time       (time) object 2000-06-16 00:00:00 ... 2001-05-16 12:00:00
  * plev       (plev) float64 1e+05 9.25e+04 8.5e+04 7e+04 ... 1e+03 500.0 100.0
  * lat        (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
  * lon        (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Dimensions without coordinates: ens_ax, bnds
Data variables:
    time_bnds  (ens_ax, time, bnds) object dask.array<chunksize=(1, 12, 2), meta=np.ndarray>
    lat_bnds   (ens_ax, lat, bnds) float64 dask.array<chunksize=(1, 96, 2), meta=np.ndarray>
    lon_bnds   (ens_ax, lon, bnds) float64 dask.array<chunksize=(1, 144, 2), meta=np.ndarray>
    ua         (ens_ax, time, plev, lat, lon) float32 dask.array<chunksize=(1, 12, 19, 96, 144), meta=np.ndarray>
Attributes: (12/49)
    Conventions:               CF-1.7 CMIP-6.2
    activity_id:               PAMIP
    branch_method:             Hybrid-restart from year 2000-04-01 of amip
    branch_time:               0.0
    branch_time_in_child:      0.0
    branch_time_in_parent:     9215.0
    ...                        ...
    title:                     NorESM2-LM output prepared for CMIP6
    variable_id:               ua
    variant_label:             r100i1p1f1
    license:                   CMIP6 model data produced by NCC is licensed u...
    cmor_version:              3.5.0
    tracking_id:               hdl:21.14100/a2aa1624-e316-436b-be2e-e20e3db1549e

### Eddy Feedback Parameter

In [33]:
nor_epfy = nor_epfy.sel( plev = nor_ua.plev.values )

nor_ds = xr.Dataset( {'ubar': nor_ua.ua.mean('lon'), 'div1': nor_epfy.epfy})
nor_ds = nor_ds.rename({'plev': 'level'})
nor_ds

<xarray.Dataset>
Dimensions:  (time: 12, level: 19, lat: 96, ens_ax: 200)
Coordinates:
  * time     (time) object 2000-06-16 00:00:00 ... 2001-05-16 12:00:00
  * level    (level) float64 1e+05 9.25e+04 8.5e+04 7e+04 ... 1e+03 500.0 100.0
  * lat      (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 84.32 86.21 88.11 90.0
Dimensions without coordinates: ens_ax
Data variables:
    ubar     (ens_ax, time, level, lat) float32 dask.array<chunksize=(1, 12, 19, 96), meta=np.ndarray>
    div1     (ens_ax, time, level, lat) float32 dask.array<chunksize=(1, 1, 19, 96), meta=np.ndarray>

In [34]:
nor_efp = ef.calculate_efp_pamip(nor_ds)
nor_efp

0.41